In [43]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import io, base64, os, json, re
import pandas as pd
import numpy as np
import datetime
from random import randint

In [44]:
# Load market data from Yahoo Finance

gspc_df = pd.read_csv('^IXIC.csv')
gspc_df['Date'] = pd.to_datetime(gspc_df['Date'])

cut_off_date = '2010-01-01'
gspc_df = gspc_df[gspc_df['Date'] >= cut_off_date]

gspc_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
9819,2010-01-04,2294.409912,2311.149902,2294.409912,2308.419922,2308.419922,1931380000
9820,2010-01-05,2307.270020,2313.729980,2295.620117,2308.709961,2308.709961,2367860000
9821,2010-01-06,2307.709961,2314.070068,2295.679932,2301.090088,2301.090088,2253340000
9822,2010-01-07,2298.090088,2301.300049,2285.219971,2300.050049,2300.050049,2270050000
9823,2010-01-08,2292.239990,2317.600098,2290.610107,2317.169922,2317.169922,2145390000


In [45]:
# take random sets of sequential rows

new_set = []
for row_set in range(0, 100000):
    if row_set%2000==0: print(row_set)
    row_quant = randint(10, 30)
    row_start = randint(0, len(gspc_df) - row_quant)
    market_subset = gspc_df.iloc[row_start:row_start+row_quant]

    close_date = max(market_subset['Date'])
    if row_set%2000 == 0: print(close_date)

    close_gap = market_subset['Close'].pct_change()
    high_gap = market_subset['High'].pct_change()
    low_gap = market_subset['Low'].pct_change()
    volume_gap = market_subset['Volume'].pct_change()
    daily_change = (market_subset['Close'] - market_subset['Open']) / market_subset['Open']
    outcome_next_day_direcion = (market_subset['Volume'].shift(-1) -  market_subset['Volume'])

    new_set.append(pd.DataFrame({
        'Sequence_ID': [row_set]*len(market_subset),
        'Close_Date': [close_date]*len(market_subset),
        'Close_Gap': close_gap,
        'High_Gap':high_gap,
        'Low_Gap':low_gap,
        'Volume_Gap':volume_gap,
        'Daily_Change':daily_change,
        'Outcome_Next_Day_Direction':outcome_next_day_direcion
    }))

0
2019-12-18 00:00:00
2000
2019-08-22 00:00:00
4000
2021-10-06 00:00:00
6000
2014-04-15 00:00:00
8000
2022-10-11 00:00:00
10000
2014-04-23 00:00:00
12000
2013-01-18 00:00:00
14000
2016-02-01 00:00:00
16000
2010-10-15 00:00:00
18000
2017-01-09 00:00:00
20000
2022-01-11 00:00:00
22000
2015-03-16 00:00:00
24000
2010-09-15 00:00:00
26000
2012-04-12 00:00:00
28000
2022-08-26 00:00:00
30000
2011-08-26 00:00:00
32000
2019-03-28 00:00:00
34000
2010-08-23 00:00:00
36000
2018-09-14 00:00:00
38000
2017-11-08 00:00:00
40000
2012-01-18 00:00:00
42000
2011-02-17 00:00:00
44000
2015-10-01 00:00:00
46000
2014-04-30 00:00:00
48000
2015-07-21 00:00:00
50000
2017-07-21 00:00:00
52000
2013-09-27 00:00:00
54000
2012-04-04 00:00:00
56000
2010-03-08 00:00:00
58000
2011-07-06 00:00:00
60000
2020-11-27 00:00:00
62000
2014-11-04 00:00:00
64000
2020-11-12 00:00:00
66000
2021-02-18 00:00:00
68000
2018-02-05 00:00:00
70000
2012-01-27 00:00:00
72000
2014-08-25 00:00:00
74000
2014-05-23 00:00:00
76000
2017-05-09 00:

In [46]:
len(market_subset)

12

In [47]:
new_set_df = pd.concat(new_set)
print(new_set_df.shape)
new_set_df = new_set_df.dropna(how='any')
print(new_set_df.shape)
new_set_df.tail(20)

(1999780, 8)
(1799780, 8)


,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction
12093,99998,2019-01-31,0.001546,0.007655,0.014434,0.047258,0.000134,-1.445000e+07
12094,99998,2019-01-31,0.007075,0.004848,-0.003486,-0.006678,0.010603,3.106400e+08
12095,99998,2019-01-31,0.010272,0.010041,0.013279,0.144532,0.003242,-6.226000e+07
12096,99998,2019-01-31,-0.019123,-0.010468,-0.016460,-0.025310,-0.012548,-1.095700e+08
12097,99998,2019-01-31,0.000771,-0.003560,-0.003808,-0.045699,-0.005081,1.310200e+08
12098,99998,2019-01-31,0.006788,-0.000831,0.011034,0.057262,0.004432,2.744000e+07
12099,99998,2019-01-31,0.012922,0.013505,0.011542,0.011343,0.005146,2.240000e+06
12100,99998,2019-01-31,-0.011051,-0.012302,-0.010806,0.000916,0.001508,-3.424200e+08
12101,99998,2019-01-31,-0.008099,0.000845,-0.003238,-0.139832,-0.008353,4.630400e+08
12102,99998,2019-01-31,0.022024,0.015372,0.007716,0.219828,0.012444,3.780800e+08


In [48]:
# Create Sequences
# Simplify Data by binning values into three groups

# Close gap
new_set_df['Close_Gap_LMH'] = pd.qcut(new_set_df['Close_Gap'], 3, labels=["L", "M", "H"])

# High Gap
new_set_df['High_Gap_LMH'] = pd.qcut(new_set_df['High_Gap'], 3, labels=["L", "M", "H"])

# Low Gap
new_set_df['Low_Gap_LMH'] = pd.qcut(new_set_df['Low_Gap'], 3, labels=["L", "M", "H"])

# Volume Gap
new_set_df['Volume_Gap_LMH'] = pd.qcut(new_set_df['Volume_Gap'], 3, labels=["L", "M", "H"])

# Daily Change
new_set_df['Daily_Change_LMH'] = pd.qcut(new_set_df['Daily_Change'], 3, labels=["L", "M", "H"])

# New Set
new_set_df = new_set_df[["Sequence_ID", "Close_Date", "Close_Gap_LMH", "Volume_Gap_LMH", "Daily_Change_LMH", "Outcome_Next_Day_Direction"]]

new_set_df['Event_Pattern'] = new_set_df["Close_Gap_LMH"].astype(str) + new_set_df["Volume_Gap_LMH"].astype(str) + new_set_df["Daily_Change_LMH"].astype(str)


In [49]:
new_set_df.tail(10)

,Sequence_ID,Close_Date,Close_Gap_LMH,Volume_Gap_LMH,Daily_Change_LMH,Outcome_Next_Day_Direction,Event_Pattern
11699,99999,2017-07-07,M,L,M,1.986880e+09,MLM
11700,99999,2017-07-07,M,H,H,-2.008850e+09,MHH
11701,99999,2017-07-07,L,L,L,3.379000e+07,LLL
11702,99999,2017-07-07,L,M,L,-1.118600e+08,LML
11703,99999,2017-07-07,H,L,H,3.331200e+08,HLH
11704,99999,2017-07-07,L,H,L,-3.987900e+08,LHL
11705,99999,2017-07-07,M,L,L,-9.206900e+08,MLL
11706,99999,2017-07-07,L,L,L,7.830100e+08,LLL
11707,99999,2017-07-07,H,H,H,1.192500e+08,HHH
11708,99999,2017-07-07,L,H,L,-2.808000e+08,LHL


In [50]:
# Reduce Set
compressed_set = new_set_df.groupby(['Sequence_ID', 'Close_Date'])['Event_Pattern'].apply(lambda x: "{%s}" % ', '.join(x)).reset_index() #Mentok

print(compressed_set.shape)
compressed_set.head()

(100000, 3)


,Sequence_ID,Close_Date,Event_Pattern
0,0,2019-12-18,"{HLH, LLM, LHL, LMH, HLM, MMM, HMM, LML, MMM, ..."
1,1,2015-09-14,"{HHM, LHL, LLM, HLH, LHL, MHH, MLL, MLH, HMH, ..."
2,2,2015-05-22,"{LML, HLH, HMM, MLM, LMM, MMM, HMH, MMM, HMH, ..."
3,3,2011-07-05,"{HHH, LLL, MLM, LML, LML, MML, LHL, MLM, LHL, ..."
4,4,2013-09-13,"{HMH, LML, HHL, HHH, MLM, MHL, HMH, HHM, MLM, ..."


In [51]:
compressed_outcomes = new_set_df.groupby(['Sequence_ID', 'Close_Date'])['Outcome_Next_Day_Direction'].mean()
compressed_outcomes = compressed_outcomes.to_frame().reset_index()
print(compressed_outcomes.shape)
compressed_outcomes.describe()

(100000, 3)


,Sequence_ID,Close_Date,Outcome_Next_Day_Direction
count,100000.000000,100000,1.000000e+05
mean,49999.500000,2016-12-01 02:19:35.616000,5.698750e+05
min,0.000000,2010-01-15 00:00:00,-6.288188e+08
25%,24999.750000,2013-07-02 00:00:00,-1.712614e+07
50%,49999.500000,2016-11-29 00:00:00,-3.687976e+05
75%,74999.250000,2020-05-07 00:00:00,1.721207e+07
max,99999.000000,2023-10-06 00:00:00,6.108862e+08
std,28867.657797,NaN,4.888776e+07


In [52]:
compressed_outcomes = pd.merge(compressed_set, compressed_outcomes, on=['Sequence_ID', 'Close_Date'], how='inner')
print(compressed_set.shape)
compressed_set.head()

(100000, 3)


,Sequence_ID,Close_Date,Event_Pattern
0,0,2019-12-18,"{HLH, LLM, LHL, LMH, HLM, MMM, HMM, LML, MMM, ..."
1,1,2015-09-14,"{HHM, LHL, LLM, HLH, LHL, MHH, MLL, MLH, HMH, ..."
2,2,2015-05-22,"{LML, HLH, HMM, MLM, LMM, MMM, HMH, MMM, HMH, ..."
3,3,2011-07-05,"{HHH, LLL, MLM, LML, LML, MML, LHL, MLM, LHL, ..."
4,4,2013-09-13,"{HMH, LML, HHL, HHH, MLM, MHL, HMH, HHM, MLM, ..."


In [53]:
# Reduce set

compressed_set['Event_Pattern'] = [''.join(e.split()).replace('{','').replace('}','') for e in compressed_set['Event_Pattern'].values]
compressed_set.head()

,Sequence_ID,Close_Date,Event_Pattern
0,0,2019-12-18,"HLH,LLM,LHL,LMH,HLM,MMM,HMM,LML,MMM,MMM,HHH,ML..."
1,1,2015-09-14,"HHM,LHL,LLM,HLH,LHL,MHH,MLL,MLH,HMH,LML,LHL,LH..."
2,2,2015-05-22,"LML,HLH,HMM,MLM,LMM,MMM,HMH,MMM,HMH,MHM,MMM,MLH"
3,3,2011-07-05,"HHH,LLL,MLM,LML,LML,MML,LHL,MLM,LHL,MLL,HLH,LH..."
4,4,2013-09-13,"HMH,LML,HHL,HHH,MLM,MHL,HMH,HHM,MLM,MMM"


In [54]:
#use last x days of data for validation
compressed_set_validation = compressed_set[compressed_set['Close_Date'] >= datetime.datetime.now() - datetime.timedelta(days=90)]

compressed_set_validation.shape

(1294, 3)

In [55]:
compressed_set = compressed_set[compressed_set['Close_Date'] < datetime.datetime.now() - datetime.timedelta(days=90)]

compressed_set.shape

(98706, 3)

In [56]:
list(compressed_set)

['Sequence_ID', 'Close_Date', 'Event_Pattern']

In [57]:
compressed_set = compressed_set[['Sequence_ID', 'Event_Pattern', 'Outcome_Next_Day_Direction']]
compressed_set_validation = compressed_set_validation[['Sequence_ID', 'Event_Pattern', 'Outcome_Next_Day_Direction']]

KeyError: "['Outcome_Next_Day_Direction'] not in index"

Keep big moves only and build outcome variable

In [58]:
compressed_set['Outcome_Next_Day_Direction'].describe()

KeyError: 'Outcome_Next_Day_Direction'

In [ ]:
print(len(compressed_set['Outcome_Next_Day_Direction']))
len(compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000])

In [ ]:
# keep only keep big / interesting moves
print('all moves:', len(compressed_set))
compressed_set = compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000]
compressed_set['Outcome_Next_Day_Direction'] = np.where((compressed_set['Outcome_Next_Day_Direction'] > 0), 1, 0)
compressed_set_validation['Outcome_Next_Day_Direction']